In [94]:
#Alle Programme instalieren / importieren
import pandas as pd
import numpy as np

from keplergl import KeplerGl 
import keplergl

from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

#https://medium.com/vis-gl/introducing-kepler-gl-for-jupyter-f72d41659fbf
#Datengrundlage
file_path = r"C:\Users\franz\OneDrive\Desktop\TUHH\3. Semester\00_TechLabs\Daten\Sauber/Unfallorte_gesamt_bereinigt.txt"
file_path1 = r"C:\Users\franz\OneDrive\Desktop\TUHH\3. Semester\00_TechLabs\Daten\csv_2016/Unfallorte_2016_LinRef.txt"
file_path2 =r"C:\Users\franz\OneDrive\Desktop\TUHH\3. Semester\00_TechLabs\Daten\csv_2016\Unfallorte_2016_LinRef.txt"
file_path3 =r"C:\Users\franz\OneDrive\Desktop\TUHH\3. Semester\00_TechLabs\Daten\keplergl (2).json"

In [61]:
#Dateneinlesen
df = pd.read_csv(file_path, sep=",")




In [62]:
df2= pd.read_csv(file_path2, sep=";")

In [63]:
#Instalaltionen
#pip install keplergl
#jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter

In [64]:
#Daten überprüfen
df.describe()




,Unnamed: 0,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstKrad,IstGkfz,IstSonstig,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,UIDENTSTLA,lat_short,long_short
count,9019.000000,9019.000000,9019.0,9019.000000,9019.000000,9019.000000,9019.000000,9019.000000,9019.000000,9019.000000,...,9019.000000,6839.000000,9019.000000,9019.000000,9.019000e+03,9019.000000,9019.000000,2.180000e+03,9019.000000,9019.000000
mean,114267.746757,114268.746757,2.0,3.336179,6.573013,348.808848,2017.525225,6.855305,13.233285,4.062535,...,0.010201,0.020471,0.088480,566613.368278,5.936381e+06,10.005960,53.572028,2.170692e+18,10.005963,53.572026
std,42798.115215,42798.115215,0.0,1.702043,4.626534,168.811413,1.118456,2.949343,4.578789,1.696236,...,0.100488,0.141615,0.284007,5714.901154,5.022940e+03,0.086317,0.045124,2.962057e+14,0.086325,0.045124
min,6022.000000,6023.000000,2.0,1.000000,1.000000,101.000000,2016.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,549085.167200,5.917209e+06,9.741474,53.397676,2.170102e+18,9.741000,53.398000
25%,97696.000000,97697.000000,2.0,2.000000,3.000000,212.000000,2017.000000,5.000000,9.000000,3.000000,...,0.000000,0.000000,0.000000,563113.830901,5.934276e+06,9.953118,53.553348,2.170503e+18,9.953000,53.553000
50%,124490.000000,124491.000000,2.0,3.000000,5.000000,320.000000,2018.000000,7.000000,14.000000,4.000000,...,0.000000,0.000000,0.000000,565984.564800,5.936364e+06,9.996110,53.572078,2.170706e+18,9.996000,53.572000
75%,128306.000000,128307.000000,2.0,5.000000,9.000000,508.000000,2019.000000,9.000000,17.000000,5.000000,...,0.000000,0.000000,0.000000,569968.091350,5.939114e+06,10.056940,53.596609,2.170920e+18,10.057000,53.597000
max,163330.000000,163331.000000,2.0,7.000000,20.000000,718.000000,2019.000000,12.000000,23.000000,7.000000,...,1.000000,1.000000,1.000000,584611.109122,5.952865e+06,10.273421,53.719033,2.171231e+18,10.273000,53.719000


In [65]:
#Long und short mit einem Rast von 100x100
Lon = np.arange(53.398, 53.719, 0.00321)
Lat = np.arange(9.741, 10.273, 0.00532) 
dfmap = np.zeros((100,100)) 

In [66]:
#The first for loop is created to go through all 2018 crime data. The second for loop is created to store crime rates by location. To do this, we go through latitude and longitude values for each incident. If lat-lon values of an incident match with one of the small cells in the 100×100 grid we created over Boston, we add 1 to that small cell. By the end of this execution, all locations will be recorded in one of the small cells in the 100×100 grid.
for a in range(len(df)):

    for b1 in range(100):
        if Lat[b1]-0.00105 <= df["lat_short"].values[a] < Lat[b1] + 0.001605:
    
         for b2 in range(100):
            if Lon[b2]-0.00119 <= df["long_short"].values[a] < Lon[b2] + 0.00266:
                dfmap[b1,b2] += 1

In [67]:
longitude_values = [Lon,]*100 
latitude_values = np.repeat(Lat,100)
dfmap.resize((10000,)) 

In [68]:
heatmap_data = {'Counts': dfmap, 'lat_short': latitude_values, 'long_short' : np.concatenate(longitude_values)} 
df1 = pd.DataFrame(data=heatmap_data) 

In [96]:
##config = {
#'version': 'v1',
'config': {
    'mapState': {
        'latitude': 53.76209132041332,
        'longitude': 9.42590232651203,
        'zoom': 12.32053899007826
    }}}


SyntaxError: unmatched '}' (<ipython-input-96-957487f8e40c>, line 8)

In [76]:
#df1['long_short'].astype('str')
#df1['lat_short'].astype('str')

In [77]:
#df1.dtypes

In [78]:
#df1['long_short'].astype('int64')
#df1['lat_short'].astype('int64')

In [79]:
df1.dtypes

Counts        float64
lat_short     float64
long_short    float64
dtype: object

In [97]:
map_1 = KeplerGl(height=800, data={"data_1": df})


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [98]:
map_1

KeplerGl(data={'data_1':       Unnamed: 0  OBJECTID  ULAND  UREGBEZ  UKREIS  UGEMEINDE  UJAHR  UMONAT  \
0    …

In [95]:
with open(file_path3,'r') as file:
    obj = json.loads(file.read())

NameError: name 'json' is not defined